In [1]:
import pandas as pd
import os
import json
import re
import numpy as np
import random
import string
import pickle
import nltk
#nltk.download()
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
from nltk import pos_tag
import gensim
from gensim import corpora, models, similarities
from sklearn.metrics.pairwise import cosine_similarity

from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.simplefilter('ignore')

In [17]:
with open("data/programming_questions.txt","r",encoding="utf-8") as file:
    que=file.read()
with open("data/programming_answers.txt","r",encoding="utf-8") as file:
    ans=file.read()

    
    

In [18]:
from nltk.tokenize import sent_tokenize
import spacy
que_sentences=que.split('\n')
facts_sentences=ans.split('\n')
print(len(que_sentences),len(facts_sentences))

40 40


In [19]:
from nltk.stem.wordnet import WordNetLemmatizer
nlp=spacy.load("en",disable=["parser","ner"])
lem=WordNetLemmatizer
def pre_process(facts):
    stop_words = stopwords.words("english")
    # Tokenlization
    facts=[x.lower() for x in facts]
    facts = [re.sub('[/(){}\[\]\|@,;]', ' ', x) for x in facts]
    facts = [re.sub('[^0-9a-z #+_]', '', x) for x in facts]
    
    facts_tokens = [nltk.word_tokenize(t) for t in facts]
    # Removing Stop Words
    facts_stop = [[t for t in tokens if (t not in stop_words) and (2 <= len(t.strip()) < 25)]
                      for tokens in facts_tokens]
    facts_lem=[[t.lemma_ for t in nlp(" ".join(tokens)) if t.text!="-PRON-"] for tokens in facts_stop]
    facts_lem=pd.Series(facts_lem)
    return facts_lem

In [20]:
facts_pp=pre_process(facts_sentences)
que_pp=pre_process(que_sentences)
print(pre_process(["give some information about internship" ])[0])
print(len(que_sentences))
print(len(que_pp),len(facts_sentences),len(facts_pp))

['give', 'information', 'internship']
40
40 40 40


In [21]:
data_tokens = pd.DataFrame({'Facts': facts_sentences,
                            'Facts_Tokens': facts_pp,
                            'Que': que_sentences,
                            'Que_Tokens': que_pp,
                           })
data_tokens.head()

,Facts,Facts_Tokens,Que,Que_Tokens
0,try this: https://code.visualstudio.com/docs/c...,"[try, https, codevisualstudiocom, docs, cpp, c...",How to install g++ correctly for VS Code?,"[install, g++, correctly, vs, code]"
1,Anyone having language doubts for now you can ...,"[anyone, language, doubt, continue, python, al...","I started learning c++, should I discontinue i...","[start, learn, c++, discontinue, learn, python..."
2,Completely you can never learn a language 100%...,"[completely, never, learn, language, 100, ever...","In how much time we can learn a language, like...","[much, time, learn, language, like, python]"
3,"In 3rd sem, there'd be this course ADP.It cove...","[3rd, sem, there, d, course, adpit, cover, htm...",will we be taught js later on?,"[teach, js, later]"
4,"Since the Data Science courses require Python,...","[since, data, science, course, require, python...",if i am already a little accustomed with pytho...,"[already, little, accustomed, python, java, li..."


In [22]:
model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
def train_model(train_data):
    """Function trains and creates Word2vec Model using parsed
    data and returns trained model"""
    #model = gensim.models.KeyedVectors.load_word2vec_format(W2V_PATH, binary=True)
    model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
    #model.train(train_data)
    #model = gensim.models.Word2Vec(train_data, min_count=1,size=150)
    return model

In [ ]:
#data_tokens['Fact_Vectors'] = None
data_tokens['Average_Pooling'] = None

facts_data = list(data_tokens['Facts_Tokens'])
que_data=list(data_tokens['Que_Tokens'])
# Train model
model_name = 'word2vec_model_.wordvectors' 
#model = train_model(facts_data)
#trained_model.save(model_name)
#print('Saved %s model successfully' % model_name)
local_model= gensim.models.Word2Vec(facts_data+que_data, min_count=1,size=300)
local_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin',
                                #lockf=1.0,
                            binary=True)
"""# Save Word2Vec model
word2vec_pickle_path =  'college_queries_word2vec_' + '.bin'
f = open(word2vec_pickle_path, 'wb')
pickle.dump(trained_model, f) 
f.close()
    
model = gensim.models.KeyedVectors.load(word2vec_pickle_path)"""

    

In [9]:
# Calculate the vectors for each question
#print(len(data_tokens))
#local_model= gensim.models.Word2Vec(facts_data, min_count=1,size=300)
#local_model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin',
#                                lockf=1.0,
 #                               binary=True)
local_model.train(que_data+facts_data,total_examples=local_model.corpus_count, epochs=15)
data_tokens['Que_Vectors'] = None
for i in range(len(data_tokens)):
            que_tokens = data_tokens['Que_Tokens'][i]
            que_vectors = []
            for token in que_tokens:
                try:
                    vector = local_model[token]
                    que_vectors.append(vector)
                except:
                    continue
            # Vectors for each tokens
            data_tokens['Que_Vectors'][i] = que_vectors
            #print(len(data_tokens['Fact_Vectors'][i]))
            # Average Pooling of all tokens
            data_tokens['Average_Pooling'][i] = list(pd.DataFrame(que_vectors).mean())

In [10]:
# Save Word2Vec model
word2vec_pickle_path =  'college_queries_word2vec_' + '.bin'
f = open(word2vec_pickle_path, 'wb')
pickle.dump(local_model, f) 
f.close()
    
#model = gensim.models.KeyedVectors.load(word2vec_pickle_path)

In [11]:
data_tokens.head()

,Facts,Facts_Tokens,Que,Que_Tokens,Average_Pooling,Que_Vectors
0,The total area of IIT Mandi is 538 acres of wh...,"[total, area, iit, mandi, 538, acre, 200, acre...",What is the area of IIT Mandi?,"[area, iit, mandi]","[-0.010416666666666666, 0.054768880208333336, ...","[[-0.072265625, 0.052490234, 0.053955078, -0.0..."
1,"On 23rd September 2012, 108 B.Tech 2nd year st...","[23rd, september, 2012, 108, btech, 2nd, year,...",When did students shift to main campus of IIT ...,"[student, shift, main, campus, iit, mandi]","[-0.026285807291666668, 0.007181803385416667, ...","[[0.036865234, 0.020141602, 0.22167969, 0.1552..."
2,The main campus of IIT Mandi is in Kamand vall...,"[main, campus, iit, mandi, kamand, valley, vil...",What is the location of IIT Mandi?,"[location, iit, mandi]","[0.024576822916666668, 0.005208333333333333, -...","[[0.032714844, -0.096191406, 0.044189453, 0.17..."
3,There is great variation in the climatic condi...,"[great, variation, climatic, condition, himach...",How is the weather at IIT Mandi?,"[weather, iit, mandi]","[-0.048502604166666664, 0.09326171875, -0.1297...","[[-0.18652344, 0.16796875, -0.30273438, -0.055..."
4,These regions enjoys a wet-sub temperate clima...,"[region, enjoy, wetsub, temperate, climate, fo...",What is the geography of Mandi?,"[geography, mandi]","[0.203125, 0.09226226806640625, -0.06604003906...","[[0.13671875, -0.0019989014, 0.033935547, 0.29..."


In [12]:
# Export as data as JSON
data_json = json.loads(data_tokens.to_json(orient='records'))

with open( 'college_query.json', 'w') as outfile:
    json.dump(data_json, outfile)

In [13]:
for i in range(1):
    stackoverflow_path = 'college_query.json'

    with open(stackoverflow_path) as file:
        reader = json.load(file)
        facts = []
        facts_tokens = []
        que=[]
        que_tokens = []
        que_vectors = []
        average_pooling = []
        for row in reader:
            facts.append(row['Facts'])
            facts_tokens.append(row['Facts_Tokens'])
            que_vectors.append(row['Que_Vectors'])
            average_pooling.append(row['Average_Pooling'])
            que.append(row['Que'])
            que_tokens.append(row['Que_Tokens'])

        data_tokens = pd.DataFrame({'Que':que,
                                    'Facts': facts,
                                    'Facts_Tokens': facts_tokens,
                                    'Que_Vectors': que_vectors,
                                    'Que_Tokens': que_tokens,
                                    'Average_Pooling': average_pooling})

data_tokens.tail()

,Que,Facts,Facts_Tokens,Que_Vectors,Que_Tokens,Average_Pooling
165,what is ubuntu?,Ubuntu is a Linux-based operating system. It i...,"[ubuntu, linuxbase, operating, system, design,...","[[0.34765625, 0.142578125, 0.251953125, 0.2714...",[ubuntu],"[0.34765625, 0.142578125, 0.251953125, 0.27148..."
166,Do I need to use Ubuntu or I can use Windows?,"There is no requirement to use ubuntu, but it ...","[requirement, use, ubuntu, preffere, use, ubun...","[[0.0072631836, 0.1005859375, -0.0600585938, 0...","[need, use, ubuntu, use, window]","[0.1358276367, 0.0135742188, 0.049609375, 0.09..."
167,Can I have both Windows and Ubuntu installed o...,Yes u can dual-boot Ubuntu and windows on a sa...,"[yes, dualboot, ubuntu, window, machine, link,...","[[0.1953125, -0.0563964844, 0.0146484375, -0.0...","[windows, ubuntu, instal, computer]","[0.0903320312, 0.0220336914, 0.1245117188, 0.1..."
168,How to dual-boot Ubuntu and Windows?,Here is a link https://www.tecmint.com/install...,"[link, https, wwwtecmintcom, still, problem, d...","[[-0.0044202921, 0.0017646761, 0.0155917592, 0...","[dualboot, ubuntu, window]","[0.1472895901, 0.0769228608, 0.0633027218, 0.0..."
169,Where can i find a learning guide for Ubuntu?,Check this documentation for Ubuntu https://ub...,"[check, documentation, ubuntu, https, ubuntuco...","[[-0.0069580078, -0.0437011719, -0.16796875, 0...","[find, learn, guide, ubuntu]","[0.0704650879, 0.0267028809, -0.0515136719, 0...."


In [14]:
def Talk_To_Jim(data, model,sentence):
    
    # Preprocessing of user input
    sentence_pp = pre_process([sentence])[0]
    cosines = []
    try:
        # Get vectors and average pooling
        fact_vectors = []
        for token in sentence_pp:
            try:
                vector = model[token]
                #print(vector.flags)
                if token=='iit' or token=='mandi':
                    vector.setflags(write=1)
                    vector/=5
                fact_vectors.append(vector)
                print("y")
            except:
                #vector = model[token]
                #fact_vectors.append(vector)
                #print("gy")
                #print(token,"not in vocab")
                continue
        #print(len(fact_vectors[0]),len(sentence_pp))
        fact_ap = list(pd.DataFrame(fact_vectors).mean())
        #print(len(fact_ap))

        # Calculate cosine similarity
        for t in data['Average_Pooling']:
            if t is not None and len(t) == len(fact_ap):
                #print("cs")
                val = cosine_similarity([fact_ap], [t])
                cosines.append(val[0][0])
            else:
                cosines.append(0)
    #except:
     #   pass
        #print(*[x for x in cosines if x>=0.63])        
    
        #else: 
        # Sort similarity
        index_s =[]
        score_s = []
        for i in range(len(cosines)):
            x = cosines[i]
            if x >= 0.65:
                #print("s")
                index_s.append(i)
                score_s.append(cosines[i])

        reply_indexes = pd.DataFrame({'index': index_s, 'score': score_s})
        reply_indexes = reply_indexes.sort_values(by="score" , ascending=False)
        
        
        # Find Top Facts and Score
        r_index = int(reply_indexes['index'].iloc[0])
        r_score = float(reply_indexes['score'].iloc[0])
        reply = str(data.iloc[:,1][r_index])
    except:
            return "Please refer GCS facebook page or ask you mentor for more info :)",999

    
        
    return reply, r_score

In [15]:
reply,score=Talk_To_Jim(data_tokens,local_model,"What is semester exchange? ")
print(reply,score)

y
y
Semester exchange offered by IIT Mandi allows students to take one or two semester(s) from the institutions with whom IIT Mandi has an MoU for such visits. 0.9999999999999998


In [16]:
model.n_similarity(["vegetable"],["fruit"])

0.56144077

In [ ]:
model.similar_by_word('hello', topn=5)

In [8]:
"""
#doc2vec 
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
documents = [TaggedDocument(data_tokens['Facts_Tokens'][i], [i]) for i in range(len(data_tokens))]
doc_model = Doc2Vec(documents, vector_size=150, window=10, min_count=1, workers=4)

from gensim.test.utils import get_tmpfile
fname = get_tmpfile("doc2vec_model")
doc_model.save(fname)
doc_model = Doc2Vec.load(fname)  # you can continue training with the loaded model!

def talk_to_doc(data,model,sentence):
    # Preprocessing of user input
    sentence_pp = pre_process([sentence])[0]
    scores=[]
    indexes=[]
    for i in range(len(data)):
        scores.append(model.n_similarity(sentence_pp,data['Facts_Tokens'][i]))
        indexes.append(i)
    reply_indexes = pd.DataFrame({'index': indexes, 'score': scores})
    reply_indexes = reply_indexes.sort_values(by="score" , ascending=False)
    print(*scores)
    # Find Top Facts and Score
    r_index = int(reply_indexes['index'].iloc[0])
    r_score = float(reply_indexes['score'].iloc[0])

    reply = str(data.iloc[:,0][r_index])
        
    return reply, r_score
        
"""  

'\n#doc2vec \nfrom gensim.test.utils import common_texts\nfrom gensim.models.doc2vec import Doc2Vec, TaggedDocument\ndocuments = [TaggedDocument(data_tokens[\'Facts_Tokens\'][i], [i]) for i in range(len(data_tokens))]\ndoc_model = Doc2Vec(documents, vector_size=150, window=10, min_count=1, workers=4)\n\nfrom gensim.test.utils import get_tmpfile\nfname = get_tmpfile("doc2vec_model")\ndoc_model.save(fname)\ndoc_model = Doc2Vec.load(fname)  # you can continue training with the loaded model!\n\ndef talk_to_doc(data,model,sentence):\n    # Preprocessing of user input\n    sentence_pp = pre_process([sentence])[0]\n    scores=[]\n    indexes=[]\n    for i in range(len(data)):\n        scores.append(model.n_similarity(sentence_pp,data[\'Facts_Tokens\'][i]))\n        indexes.append(i)\n    reply_indexes = pd.DataFrame({\'index\': indexes, \'score\': scores})\n    reply_indexes = reply_indexes.sort_values(by="score" , ascending=False)\n    print(*scores)\n    # Find Top Facts and Score\n    r_in

In [144]:
print(pre_process(["Where is IIT Mandi located?"])[0])

['iit', 'mandi', 'located']
